In [19]:
import numpy as np
import os, sys, glob
import matplotlib.pyplot as plt
import open3d as o3d
from scipy.stats import chisquare
import pandas as pd

basedir = os.path.dirname(os.getcwd())
_py = os.path.join(basedir, 'py')
_data = os.path.join(basedir, 'data')
_images = os.path.join(basedir, 'images')

sys.path.insert(1, _py)
import loads
import lia
import lad
import figures

from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

%matplotlib qt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Params init

Initialize parameters

In [7]:
mockname = 'test'

## Prepare data

Convert Blensor output txt files that have fake numpy extension to real npy

In [8]:
loads.numpy2npy(mockname)

Next wee create the module to segmentate trees. This will be tuned acordingly for each data set.

In [24]:
def segtree(df, show=False):

    trees = {}

    if show:
        plt.figure(figsize=(14, 8))

    # centres
    x, y = [0], [0]
    num = 0
    dx, dy = 5, 5

    # resdir_name = 'toy_trees'
    # resdir = os.path.join(datapath, resdir_name)
    # if not os.path.exists(resdir):
    #     os.makedirs(resdir)

    for i in x:
        for j in y:
            
            keep = np.ones(len(df['x']), dtype=bool)
            keep &= (df['x'] < i+dx) & (df['x'] > i-dx)
            keep &= (df['y'] < j+dy) & (df['y'] > j-dy)

            trees['tree_%s' %(str(num))] = keep
            
            if show:
                plt.scatter(df['x'][leaves & keep], df['y'][leaves & keep], s=0.5, label=num)
                        
            num += 1

    if show:
        plt.legend()
    
    return trees


In [26]:
# load data into a pandas data frame
df = loads.npy2pandas(mockname)
# extract leaves. Boolean array output
leaves = loads.extract_leaves(df, show=False)
# extract trees. Dictionary with boolean arrays output
trees = segtree(df)

s0200000.npy (22500, 16) (22500, 3)
s0300000.npy (22500, 16) (22500, 3)
s0700000.npy (22500, 16) (22500, 3)
s0600000.npy (22500, 16) (22500, 3)
s0500000.npy (22500, 16) (22500, 3)
s0100000.npy (22500, 16) (22500, 3)
s0400000.npy (22500, 16) (22500, 3)


In [ ]:
# show the point cloud from leaves of firs tree only
keep = (trees['tree_0']) & (leaves)
loads.showPCfromDF(df[keep])

## Compute the `Leaf Inclination Angle` (LIA) for each tree

In [37]:
for key, val in trees.items():

    keep = (val) & (leaves)
    df_ = df[['x', 'y', 'z']][keep]
    points = loads.DF2array(df[['x', 'y', 'z']][keep])
    print(points.shape)

(68192, 3)


In [34]:
df_

,x,y,z
54,3.529133,2.096912,6.296125
55,3.551827,2.022412,6.300459
56,3.509961,2.093048,6.408172
57,3.556426,1.964666,6.375213
58,3.586435,1.872954,6.369163
...,...,...,...
157468,2.230066,2.795502,8.873904
157469,2.205568,2.812099,8.913189
157470,2.181042,2.828716,8.952522
157471,2.156486,2.845352,8.991902


In [12]:
df = loads.npy2pandas(mockname)

s0200000.npy (22500, 16) (22500, 3)
s0300000.npy (22500, 16) (22500, 3)
s0700000.npy (22500, 16) (22500, 3)
s0600000.npy (22500, 16) (22500, 3)
s0500000.npy (22500, 16) (22500, 3)
s0100000.npy (22500, 16) (22500, 3)
s0400000.npy (22500, 16) (22500, 3)


In [17]:
leaves = loads.extract_leaves(df, show=False)

In [ ]:
# Here we show how some of the point cloud parameters looks like.
cols = ['x','y']

plt.figure(figsize=(18, 4*3))

for num, i in enumerate(cols):
    
    plt.subplot(3,3,num+1)
    plt.hist(scan[i][leaves], 80, alpha=0.5, label=i)
    plt.legend()
    
plt.show()

plt.figure(figsize=(14, 8))
plt.scatter(scan['x'][leaves], scan['y'][leaves], s=1)

In [ ]:
np.sum(leaves)

In [ ]:
#
plt.figure(figsize=(14, 8))

# x = np.linspace(0, 16, 3)
# y = np.array([0, 8])

# centres
x, y = [0], [0]

trees = {}
num = 0
# dx, dy = 4, 4
dx, dy = 5, 5

resdir_name = 'toy_trees'
resdir = os.path.join(datapath, resdir_name)
if not os.path.exists(resdir):
    os.makedirs(resdir)

for i in x:
    for j in y:
        
        keep = np.ones(len(scan['x']), dtype=bool)
        keep &= (scan['x'] < i+dx) & (scan['x'] > i-dx)
        keep &= (scan['y'] < j+dy) & (scan['y'] > j-dy)
        
        plt.scatter(scan['x'][leaves & keep], scan['y'][leaves & keep], s=0.5, label=num)
        
        np.save(os.path.join(resdir, 'tree_%s' %(str(num))), scan[leaves & keep])
        
        num += 1
        
plt.legend()   

# DEV ZONE...

# define the path of the data stored
mockname = 'single'
datapath = os.path.join(_data, mockname, 'toy_trees')
file = glob.glob(os.path.join(datapath, 'tree*.npy'))
print(file)
df = np.load(file[0])

In [ ]:
mockname = 'single_tmp'
datapath = os.path.join(_data, mockname)

# read the numpy files
files = {}
plt.figure(figsize=(18,14))
for file in glob.glob(os.path.join(datapath, 's*.npy')):
    filename = file.split('/')[-1]
    df = np.load(file)
    # df = df[::1]
    yaw = df.T[1]
    pitch = df.T[2]
    x, y = df.T[5], df.T[6]

    
    plt.subplot(3,2,1)
    plt.title('yaw')
    #np.linspace(2.8, 3.5, 40)
    plt.hist(np.rad2deg(yaw), bins=40, histtype='step', lw=2, label=filename)
    plt.legend()
    plt.subplot(3,2,2)
    plt.title('pitch')
    #np.linspace(-0.35, 0.3, 40)
    plt.hist(np.rad2deg(pitch), bins=40, histtype='step', lw=2, label=filename)
    plt.subplot(3,2,3)
    plt.title('x')
    plt.hist(x, bins=40, histtype='step', lw=2, label=filename)
    plt.subplot(3,2,4)
    plt.title('y')
    plt.hist(y, bins=40, histtype='step', lw=2, label=filename)
    plt.subplot(3,2,5)
    uv = lad.beam_direction(yaw, pitch, scan_inc=90)
    beam_angs = []
    for i in uv:
        beam_angs.append(lad.vecttoangle([0, 0, 1], i))
    plt.hist(np.array(beam_angs), 40, histtype='step', label='pre correction')
    plt.legend()
    plt.subplot(3,2,6)
    plt.hist(uv.T[0], bins=40, histtype='step', lw=2, label=filename+' x')
    plt.hist(uv.T[1], bins=40, histtype='step', lw=2, label=filename+' y')
    plt.hist(uv.T[2], bins=40, histtype='step', lw=2, label=filename+' z')
    plt.legend()


In [ ]:
colors = ['b', 'cyan', 'orange', 'yellow']
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')

for num, file in enumerate(glob.glob(os.path.join(datapath, 's1*.npy'))):
    
    filename = file.split('/')[-1]
    df = np.load(file)
    df = df[::10]
    print(df.shape)
    # yaw = df.T[1]
    # pitch = df.T[2]
    x, y, z = df.T[5], df.T[6], df.T[7]
    # sensor coordinates
    spos = os.path.join(datapath, 'scanner_pos.txt')
    scan = lad.laod_scan_pos(spos)
    id = [i.decode("utf-8") for i in scan['scan']]
    keep = np.array(id) == filename[:2]
    _, sx, sy, sz = scan[keep][0]
    print(scan[keep][0])

    ax.scatter3D(x, y, z, c='g', s=1)
    ax.scatter3D(sx, sy, sz, c='r', s=10)

    p2 = [sx, sy, sz]
    res = 2

    for i, j, k in zip(x, y, z):

        p1 = [i, j, k]

        px, py, pz = lad.line2points_vect(p1, p2, res=res)
        # ax.scatter3D(px, py, pz, c='k', s=1)
        ax.plot3D([i, sx], [j, sy], [k, sz], lw=0.1, color=colors[num])


In [ ]:
# get true leag-angle-distribution
file = "../data/single_57_inv/mesh.ply"
mesh = o3d.io.read_triangle_mesh(file)
# 
mesh.compute_triangle_normals()
mesh.compute_vertex_normals()
# print(mesh.cluster_connected_triangles())
# print("Computing normal and rendering it.")
# 
print(np.asarray(mesh.triangle_normals))
print(np.asarray(mesh.triangles))
o3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True)


In [ ]:
# get leaf area
file = "../data/spherical_02/mesh.ply"
mesh = o3d.io.read_triangle_mesh(file)
cidx, nt, area = mesh.cluster_connected_triangles()
la = np.array(area)[np.array(nt) == 4][0]
np.round(la, 4)

In [ ]:
len(area)

In [ ]:
set(np.round(np.array(area), 5))

In [ ]:
np.array(area)[np.array(nt) == 4][0]